## A demo bayesian network

In [132]:
from collections import defaultdict

class Factor:
    def __init__(self, variables, prob_table):
        """
        Variable: dict, {varaibel: value_range}
        prob_table: dict, the probability table 
        of the variable, index sorted {(v1, v2, v3):prob}
        """
        self.variables = variables
        self.prob_table = prob_table
        
        
    def prob(self, dict_assignment_vals):
        """
        parant_vals: {var1:value, var2: value}
        """
        val_key = tuple((dict_assignment_vals[var] 
                         for var in self.variables.keys()))
        return self.prob_table[val_key]
    
    
def dfs(variables, index, running_res, results):
    if len(running_res) == len(variables):
        results.append(running_res.copy())
        return
        
    for i, var in enumerate(variables):
        if i != index: continue        
        for val in range(1, variables[var]+1):
            running_res[var] = val
            dfs(variables, index+1, running_res, results)
            running_res.pop(var)
    
def factor_product(f1, f2):
    """
    factor product operation on two factors
    """
    common_vars = set(f1.variables.keys()) & set(f2.variables.keys())
    disjoint_vars_f1 = set(f1.variables.keys()) - set(f2.variables.keys())
    disjoint_vars_f2 = set(f2.variables.keys()) - set(f1.variables.keys())
    
    # merge new variables
    new_vars = {}
    for var in disjoint_vars_f1 | common_vars:
        new_vars[var] = f1.variables[var]
    for var in disjoint_vars_f2:
        new_vars[var] = f2.variables[var]
    
    # create all possible new assignments
    all_assignments = []
    dfs(new_vars, 0, {}, all_assignments)
    
    #print(all_assignments)
    
    # compute product of probability
    new_prob_table = {}
    for an_assignment in all_assignments:
        f1_assignments, f2_assignments = {}, {}
        for var, value in an_assignment.items():
            if var in disjoint_vars_f1:
                f1_assignments[var] = value
                continue
            if var in disjoint_vars_f2:
                f2_assignments[var] = value
                continue
            f1_assignments[var] = value
            f2_assignments[var] = value
        #print(f1_assignments, f2_assignments)    
        
        product_prob = f1.prob(f1_assignments) * f2.prob(f2_assignments)
        #print(product_prob)
        
        val_key = tuple((an_assignment[_var] for _var in new_vars.keys()))
        new_prob_table[val_key] = product_prob
    return Factor(new_vars, new_prob_table)

def factor_marginalize(factor, variables_to_sumout):
    """
    Marginalize a factor given a list of variables
    """
    for var in variables_to_sumout:
        if not var in factor.variables:
            print("Invalid variables found, please make \
                  sure the operation is valid.")
            return factor
    remaining_vars = set(factor.variables.keys()) - set(variables_to_sumout)
    remaining_vars = {v:factor.variables[v] for v in remaining_vars}
    
#     if not remaining_vars:
#         return Factor({"None":0}, {("None",):1})
        
    remaining_assignments = []
    dfs(remaining_vars, 0, {}, remaining_assignments)
    
    variables_to_sumout = {v:factor.variables[v] for v in variables_to_sumout}
    sumout_assignments = []
    dfs(variables_to_sumout, 0, {}, sumout_assignments)
    
    
    # for each remaining assignment, 
    new_prob_table = {}
    for r_assign in remaining_assignments:
        margin_sum = sum([factor.prob({**r_assign, **sumout_assign}) 
                          for sumout_assign in sumout_assignments])
        value_idx = val_key = tuple((r_assign[_var] for _var in 
                                     remaining_vars.keys()))
        new_prob_table[value_idx] = margin_sum
        
    return Factor(remaining_vars, new_prob_table)

### Replecate tutorial case linear Bayesian Network

In [150]:
phi1 = Factor({1:2}, {(1,):0.11, (2,):0.89})
phi2 = Factor({2:2, 1:2}, {(1,1):0.59, (2,1):0.41, (1,2):0.22, (2,2):0.78})
phi3 = Factor({3:2, 2:2}, {(1,1):0.39, (2,1):0.61, (1,2):0.06, (2,2):0.94})


print("Product of Phi1 and Phi2: ")
print(factor_product(phi1, phi2).variables)


print("\nProbability table")

for _assign, _prob in factor_product(phi1, phi2).prob_table.items():
    print("assignment: {}, probability: {}".format(_assign, _prob))
      
print("\nMarginalization result phi2, [2]",
    factor_marginalize(phi2, [2]).variables, 
    factor_marginalize(phi2, [2]).prob_table, 
    sep="\n")

Product of Phi1 and Phi2: 
{1: 2, 2: 2}

Probability table
assignment: (1, 1), probability: 0.0649
assignment: (1, 2), probability: 0.045099999999999994
assignment: (2, 1), probability: 0.1958
assignment: (2, 2), probability: 0.6942

Marginalization result phi2, [2]
{1: 2}
{(1,): 1.0, (2,): 1.0}


### Question BN 

- Values are indexed from 1 to 2 (instead of from 0, so as to align tutorial and the assignment question), so Difficulty has range set {1, 2} instead of {0, 1}

In [152]:
phi1 = Factor({1:2}, {(1,):0.6, (2,):0.4})
phi2 = Factor({2:2}, {(1,):0.7, (2,):0.3})
phi3 = Factor({3:3, 1:2, 2:2}, {(1,1,1):0.3, (2,1,1):0.4, (3,1,1):0.3, 
                                (1,1,2):0.05, (2,1,2):0.25, (3,1,2):0.7,
                                (1,2,1):0.9, (2,2,1):0.08, (3,2,1):0.02,
                                (1,2,2):0.5, (2,2,2):0.3, (3,2,2):0.2
                               })
phi4 = Factor({4:2, 2:2}, {(1,1):0.95, (2,1):0.05, (1,2):0.2, (2,2):0.8})
phi5 = Factor({5:2, 3:3}, {(1,1):0.1, (2,1):0.9,
                           (1,2):0.4, (2,2):0.6,
                           (1,3):0.99, (2,3):0.01})

factors = [phi1, phi2, phi3, phi4, phi5]
# get joint probability
product = phi1
for i in range(1, len(factors)):
    product = factor_product(product, factors[i])
    
    
print("\nJoint probability table:")
for _assign, _prob in product.prob_table.items():
    print("assignment: {}, probability: {}".format(_assign, _prob))

print("\nMarginalize joint probability")
print(factor_marginalize(product, [1,2,3,4,5]).prob_table)

print("\nMarginzailze joint probability sum out [phi2,3,4 and 5] ")
print(factor_marginalize(product,[2,3,4,5]).prob_table)


#print("", factor_marginalize(phi3,[3]).prob_table)


Joint probability table:
assignment: (1, 1, 1, 1, 1), probability: 0.011970000000000001
assignment: (1, 1, 1, 1, 2), probability: 0.10773
assignment: (1, 1, 1, 2, 1), probability: 0.00063
assignment: (1, 1, 1, 2, 2), probability: 0.0056700000000000006
assignment: (1, 1, 2, 1, 1), probability: 0.06384
assignment: (1, 1, 2, 1, 2), probability: 0.09576
assignment: (1, 1, 2, 2, 1), probability: 0.0033600000000000006
assignment: (1, 1, 2, 2, 2), probability: 0.00504
assignment: (1, 1, 3, 1, 1), probability: 0.118503
assignment: (1, 1, 3, 1, 2), probability: 0.001197
assignment: (1, 1, 3, 2, 1), probability: 0.006237
assignment: (1, 1, 3, 2, 2), probability: 6.3e-05
assignment: (1, 2, 1, 1, 1), probability: 0.00018
assignment: (1, 2, 1, 1, 2), probability: 0.00162
assignment: (1, 2, 1, 2, 1), probability: 0.00072
assignment: (1, 2, 1, 2, 2), probability: 0.00648
assignment: (1, 2, 2, 1, 1), probability: 0.0036
assignment: (1, 2, 2, 1, 2), probability: 0.005399999999999999
assignment: (1, 2,